# Sesión 4 – Comparación entre SLM y LLM

Comparar la latencia y la calidad de las respuestas de muestra entre un Modelo de Lenguaje Pequeño y un modelo más grande ejecutado a través de Foundry Local.


## ⚡ Inicio Rápido

**Configuración Optimizada para Memoria (Actualizada):**
1. Los modelos seleccionan automáticamente variantes de CPU (funciona en cualquier hardware).
2. Utiliza `qwen2.5-3b` en lugar de 7B (ahorra ~4GB de RAM).
3. Detección automática de puertos (sin configuración manual).
4. RAM total necesaria: ~8GB recomendados (modelos + sistema operativo).

**Configuración en Terminal (30 segundos):**
```bash
foundry service start
foundry model run phi-4-mini
foundry model run qwen2.5-3b
```

¡Luego ejecuta este notebook! 🚀


### Explicación: Instalación de Dependencias
Instala los paquetes mínimos (`foundry-local-sdk`, `openai`, `numpy`) necesarios para solicitudes de tiempo y chat. Es seguro ejecutarlo de nuevo de manera idempotente.


# Escenario
Compara un Modelo de Lenguaje Pequeño (SLM) representativo con un modelo más grande utilizando un único mensaje para ilustrar los compromisos:
- **Diferencia de latencia** (segundos de reloj)
- **Uso de tokens** (si está disponible) como indicador de rendimiento
- **Muestra de salida cualitativa** para una evaluación rápida
- **Cálculo de aceleración** para cuantificar las mejoras de rendimiento

**Variables de entorno:**
- `SLM_ALIAS` - Modelo de lenguaje pequeño (por defecto: phi-4-mini, ~4GB RAM)
- `LLM_ALIAS` - Modelo de lenguaje más grande (por defecto: qwen2.5-7b, ~7GB RAM)
- `COMPARE_PROMPT` - Mensaje de prueba para la comparación
- `COMPARE_RETRIES` - Intentos de reintento para mayor resiliencia (por defecto: 2)
- `FOUNDRY_LOCAL_ENDPOINT` - Sobrescribir el punto de acceso del servicio (detectado automáticamente si no se establece)

**Cómo funciona (Patrón oficial del SDK):**
1. **FoundryLocalManager** inicializa y gestiona el servicio Foundry Local
2. El servicio se inicia automáticamente si no está en ejecución (no se necesita configuración manual)
3. Los modelos se resuelven automáticamente de alias a IDs concretos
4. Se seleccionan variantes optimizadas para hardware (CUDA, NPU o CPU)
5. El cliente compatible con OpenAI realiza las completaciones de chat
6. Se capturan métricas: latencia, tokens, calidad de salida
7. Los resultados se comparan para calcular la relación de aceleración

Esta microcomparación ayuda a decidir cuándo es justificable redirigir a un modelo más grande para tu caso de uso.

**Referencia del SDK:** 
- SDK de Python: https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local
- Utilidades del taller: Utiliza el patrón oficial de ../samples/workshop_utils.py

**Beneficios clave:**
- ✅ Descubrimiento e inicialización automática del servicio
- ✅ Inicio automático del servicio si no está en ejecución
- ✅ Resolución y almacenamiento en caché de modelos integrado
- ✅ Optimización de hardware (CUDA/NPU/CPU)
- ✅ Compatibilidad con el SDK de OpenAI
- ✅ Manejo robusto de errores con reintentos
- ✅ Inferencia local (no se requiere API en la nube)


## 🚨 Prerrequisitos: ¡Foundry Local debe estar en ejecución!

**Antes de ejecutar este notebook**, asegúrate de que el servicio Foundry Local esté configurado:

### Comandos de inicio rápido (Ejecutar en la terminal):

```bash
# 1. Start the Foundry Local service
foundry service start

# 2. Load the default models used in this comparison (CPU-optimized)
foundry model run phi-4-mini
foundry model run qwen2.5-3b

# 3. Verify models are loaded
foundry model ls

# 4. Check service health
foundry service status
```

### Modelos alternativos (si los predeterminados no están disponibles):

```bash
# Even smaller alternatives (if memory is very limited)
foundry model run phi-3.5-mini
foundry model run qwen2.5-0.5b

# Or update the environment variables in this notebook:
# SLM_ALIAS = 'phi-3.5-mini'
# LLM_ALIAS = 'qwen2.5-1.5b'  # Or qwen2.5-0.5b for minimum memory
```

⚠️ **Si omites estos pasos**, aparecerá `APIConnectionError` al ejecutar las celdas del notebook a continuación.


In [29]:
# Install dependencies
!pip install -q foundry-local-sdk openai numpy requests

### Explicación: Importaciones principales
Incluye utilidades de tiempo y clientes de Foundry Local / OpenAI utilizados para obtener información del modelo y realizar completaciones de chat.


In [30]:
import os, time, json
from foundry_local import FoundryLocalManager
from openai import OpenAI
import sys
sys.path.append('../samples')
from workshop_utils import get_client, chat_once

### Explicación: Alias y Configuración de Prompt
Define alias configurables según el entorno para modelos pequeños y grandes, además de un prompt de comparación. Ajusta las variables de entorno para experimentar con diferentes familias de modelos o tareas.


In [31]:
# Default to CPU models for better memory efficiency
SLM = os.getenv('SLM_ALIAS', 'phi-4-mini')  # Auto-selects CPU variant
LLM = os.getenv('LLM_ALIAS', 'qwen2.5-3b')  # Smaller LLM, more memory-friendly
PROMPT = os.getenv('COMPARE_PROMPT', 'List 5 benefits of local AI inference.')
# Endpoint is now managed by FoundryLocalManager - it auto-detects or can be overridden
ENDPOINT = os.getenv('FOUNDRY_LOCAL_ENDPOINT', None)

### 💡 Configuración Optimizada para Memoria

**Este cuaderno utiliza modelos eficientes en memoria por defecto:**
- `phi-4-mini` → ~4GB de RAM (Foundry Local selecciona automáticamente la variante para CPU)
- `qwen2.5-3b` → ~3GB de RAM (en lugar de 7B que necesita ~7GB+)

**Detección Automática de Puertos:**
- Foundry Local puede usar diferentes puertos (comúnmente 55769 o 59959)
- La celda de diagnóstico a continuación detecta automáticamente el puerto correcto
- ¡No se necesita configuración manual!

**Si tienes menos de 8GB de RAM, utiliza modelos aún más pequeños:**
```python
SLM = 'phi-3.5-mini'      # ~2GB
LLM = 'qwen2.5-0.5b'      # ~500MB
```


In [32]:
# Display current configuration
print("="*60)
print("CURRENT CONFIGURATION")
print("="*60)
print(f"SLM Model:     {SLM}")
print(f"LLM Model:     {LLM}")
print(f"SDK Pattern:   FoundryLocalManager (official)")
print(f"Endpoint:      {ENDPOINT or 'Auto-detect'}")
print(f"Test Prompt:   {PROMPT[:50]}...")
print(f"Retry Count:   2")
print("="*60)
print("\n💡 Using official Foundry SDK pattern from workshop_utils")
print("   → FoundryLocalManager handles service lifecycle")
print("   → Automatic model resolution and hardware optimization")
print("   → OpenAI-compatible API for inference")

CURRENT CONFIGURATION
SLM Model:     phi-4-mini
LLM Model:     qwen2.5-7b
SDK Pattern:   FoundryLocalManager (official)
Endpoint:      Auto-detect
Test Prompt:   List 5 benefits of local AI inference....
Retry Count:   2

💡 Using official Foundry SDK pattern from workshop_utils
   → FoundryLocalManager handles service lifecycle
   → Automatic model resolution and hardware optimization
   → OpenAI-compatible API for inference


### Explicación: Ayudantes de Ejecución (Patrón del SDK de Foundry)
Utiliza el patrón oficial del SDK Local de Foundry, tal como se documenta en los ejemplos del Workshop:

**Enfoque:**
- **FoundryLocalManager** - Inicializa y gestiona el servicio Local de Foundry
- **Detección Automática** - Descubre automáticamente el endpoint y maneja el ciclo de vida del servicio
- **Resolución de Modelos** - Resuelve alias a IDs completos de modelos (por ejemplo, phi-4-mini → phi-4-mini-instruct-cpu)
- **Optimización de Hardware** - Selecciona la mejor variante para el hardware disponible (CUDA, NPU o CPU)
- **Cliente OpenAI** - Configurado con el endpoint del administrador para acceso a la API compatible con OpenAI

**Características de Resiliencia:**
- Lógica de reintento con retroceso exponencial (configurable mediante el entorno)
- Inicio automático del servicio si no está en ejecución
- Verificación de conexión después de la inicialización
- Manejo de errores de forma elegante con informes detallados
- Caché de modelos para evitar inicializaciones repetidas

**Estructura de Resultados:**
- Medición de latencia (tiempo de reloj)
- Seguimiento del uso de tokens (si está disponible)
- Salida de muestra (truncada para facilitar la lectura)
- Detalles de errores para solicitudes fallidas

Este patrón aprovecha el módulo workshop_utils, que sigue el patrón oficial del SDK.

**Referencia del SDK:**
- Repositorio Principal: https://github.com/microsoft/Foundry-Local
- SDK de Python: https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local
- Workshop Utils: ../samples/workshop_utils.py


In [39]:
def setup(alias: str, endpoint: str = None, retries: int = 3):
    """
    Initialize a Foundry Local model connection using official SDK pattern.
    
    This follows the workshop_utils pattern which uses FoundryLocalManager
    to properly initialize the Foundry Local service and resolve models.
    
    Args:
        alias: Model alias (e.g., 'phi-4-mini', 'qwen2.5-3b')
        endpoint: Optional endpoint override (usually auto-detected)
        retries: Number of connection attempts (default: 3)
    
    Returns:
        tuple: (manager, client, model_id, metadata) or (None, None, alias, error_metadata) if failed
    """
    import time
    
    last_err = None
    current_delay = 2  # seconds
    
    for attempt in range(1, retries + 1):
        try:
            print(f"[Init] Connecting to '{alias}' (attempt {attempt}/{retries})...")
            
            # Use the workshop utility which follows the official SDK pattern
            manager, client, model_id = get_client(alias, endpoint=endpoint)
            
            print(f"[OK] Connected to '{alias}' -> {model_id}")
            print(f"     Endpoint: {manager.endpoint}")
            
            return manager, client, model_id, {
                'endpoint': manager.endpoint,
                'resolved': model_id,
                'attempts': attempt,
                'status': 'success'
            }
            
        except Exception as e:
            last_err = e
            error_msg = str(e)
            
            # Provide helpful error messages
            if "Connection error" in error_msg or "connection refused" in error_msg.lower():
                print(f"[ERROR] Cannot connect to Foundry Local service")
                print(f"        → Is the service running? Try: foundry service start")
                print(f"        → Is the model loaded? Try: foundry model run {alias}")
            elif "not found" in error_msg.lower():
                print(f"[ERROR] Model '{alias}' not found in catalog")
                print(f"        → Available models: Run 'foundry model ls' in terminal")
                print(f"        → Download model: Run 'foundry model download {alias}'")
            else:
                print(f"[ERROR] Setup failed: {type(e).__name__}: {error_msg}")
            
            if attempt < retries:
                print(f"[Retry] Waiting {current_delay:.1f}s before retry...")
                time.sleep(current_delay)
                current_delay *= 2  # Exponential backoff
    
    # All retries failed - provide actionable guidance
    print(f"\n❌ Failed to initialize '{alias}' after {retries} attempts")
    print(f"   Last error: {type(last_err).__name__}: {str(last_err)}")
    print(f"\n💡 Troubleshooting steps:")
    print(f"   1. Ensure Foundry Local service is running:")
    print(f"      → foundry service status")
    print(f"      → foundry service start (if not running)")
    print(f"   2. Ensure model is loaded:")
    print(f"      → foundry model run {alias}")
    print(f"   3. Check available models:")
    print(f"      → foundry model ls")
    print(f"   4. Try alternative models if '{alias}' isn't available")
    
    return None, None, alias, {
        'error': f"{type(last_err).__name__}: {str(last_err)}",
        'endpoint': endpoint or 'auto-detect',
        'attempts': retries,
        'status': 'failed'
    }


def run(client, model_id: str, prompt: str, max_tokens: int = 180, temperature: float = 0.5):
    """
    Run inference with the configured model using OpenAI SDK.
    
    Args:
        client: OpenAI client instance (configured for Foundry Local)
        model_id: Model identifier (resolved from alias)
        prompt: Input prompt
        max_tokens: Maximum response tokens (default: 180)
        temperature: Sampling temperature (default: 0.5)
    
    Returns:
        dict: Response with timing, tokens, and content
    """
    import time
    
    start = time.time()
    
    try:
        response = client.chat.completions.create(
            model=model_id,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            temperature=temperature
        )
        
        elapsed = time.time() - start
        
        # Extract response details
        content = response.choices[0].message.content
        
        # Try to extract token usage from multiple possible locations
        usage_info = {}
        if hasattr(response, 'usage') and response.usage:
            usage_info['prompt_tokens'] = getattr(response.usage, 'prompt_tokens', None)
            usage_info['completion_tokens'] = getattr(response.usage, 'completion_tokens', None)
            usage_info['total_tokens'] = getattr(response.usage, 'total_tokens', None)
        
        # Calculate approximate token count if API doesn't provide it
        # Rough estimate: ~4 characters per token for English text
        if not usage_info.get('total_tokens'):
            estimated_prompt_tokens = len(prompt) // 4
            estimated_completion_tokens = len(content) // 4
            estimated_total = estimated_prompt_tokens + estimated_completion_tokens
            usage_info['estimated_tokens'] = estimated_total
            usage_info['estimated_prompt_tokens'] = estimated_prompt_tokens
            usage_info['estimated_completion_tokens'] = estimated_completion_tokens
        
        return {
            'status': 'success',
            'content': content,
            'elapsed_sec': elapsed,
            'tokens': usage_info.get('total_tokens') or usage_info.get('estimated_tokens'),
            'usage': usage_info,
            'model': model_id
        }
        
    except Exception as e:
        elapsed = time.time() - start
        return {
            'status': 'error',
            'error': f"{type(e).__name__}: {str(e)}",
            'elapsed_sec': elapsed,
            'model': model_id
        }


print("✅ Execution helpers defined: setup(), run()")
print("   → Uses workshop_utils for proper SDK integration")
print("   → setup() initializes with FoundryLocalManager")
print("   → run() executes inference via OpenAI-compatible API")
print("   → Token counting: Uses API data or estimates if unavailable")

✅ Execution helpers defined: setup(), run()
   → Uses workshop_utils for proper SDK integration
   → setup() initializes with FoundryLocalManager
   → run() executes inference via OpenAI-compatible API
   → Token counting: Uses API data or estimates if unavailable


### Explicación: Autoevaluación previa al vuelo
Realiza una verificación de conectividad ligera utilizando FoundryLocalManager para ambos modelos. Esto verifica:
- Que el servicio sea accesible
- Que los modelos puedan inicializarse
- Que los alias se resuelvan en IDs de modelo reales
- Que la conexión sea estable antes de ejecutar la comparación

La función setup() utiliza el patrón oficial del SDK de workshop_utils.


In [34]:
# Simplified diagnostic: Just verify service is accessible
import requests

def check_foundry_service():
    """Quick diagnostic to verify Foundry Local is running."""
    # Try common ports
    endpoints_to_try = [
        "http://localhost:59959",
        "http://127.0.0.1:59959", 
        "http://localhost:55769",
        "http://127.0.0.1:55769",
    ]
    
    print("[Diagnostic] Checking Foundry Local service...")
    
    for endpoint in endpoints_to_try:
        try:
            response = requests.get(f"{endpoint}/health", timeout=2)
            if response.status_code == 200:
                print(f"✅ Service is running at {endpoint}")
                
                # Try to list models
                try:
                    models_response = requests.get(f"{endpoint}/v1/models", timeout=2)
                    if models_response.status_code == 200:
                        models_data = models_response.json()
                        model_count = len(models_data.get('data', []))
                        print(f"✅ Found {model_count} models available")
                        if model_count > 0:
                            print("   Models:", [m.get('id', 'unknown') for m in models_data.get('data', [])[:5]])
                except Exception as e:
                    print(f"⚠️  Could not list models: {e}")
                
                return endpoint
        except requests.exceptions.ConnectionError:
            continue
        except Exception as e:
            print(f"⚠️  Error checking {endpoint}: {e}")
    
    print("\n❌ Foundry Local service not found!")
    print("\n💡 To fix this:")
    print("   1. Open a terminal")
    print("   2. Run: foundry service start")
    print("   3. Run: foundry model run phi-4-mini")
    print("   4. Run: foundry model run qwen2.5-3b")
    print("   5. Re-run this notebook")
    return None

# Run diagnostic
discovered_endpoint = check_foundry_service()

if discovered_endpoint:
    print(f"\n✅ Service detected (will be managed by FoundryLocalManager)")
else:
    print(f"\n⚠️  No service detected - FoundryLocalManager will attempt to start it")

[Diagnostic] Checking Foundry Local service...

❌ Foundry Local service not found!

💡 To fix this:
   1. Open a terminal
   2. Run: foundry service start
   3. Run: foundry model run phi-4-mini
   4. Run: foundry model run qwen2.5-3b
   5. Re-run this notebook

⚠️  No service detected - FoundryLocalManager will attempt to start it


In [35]:
# Quick Fix: Start service and load models from notebook
# Uncomment the commands you need:

# !foundry service start
# !foundry model run phi-4-mini
# !foundry model run qwen2.5-3b
# !foundry model ls

print("⚠️  The commands above are commented out.")
print("Uncomment them if you want to start the service from the notebook.")
print("")
print("💡 Recommended: Run these commands in a separate terminal instead:")
print("   foundry service start")
print("   foundry model run phi-4-mini")
print("   foundry model run qwen2.5-3b")

⚠️  The commands above are commented out.
Uncomment them if you want to start the service from the notebook.

💡 Recommended: Run these commands in a separate terminal instead:
   foundry service start
   foundry model run phi-4-mini
   foundry model run qwen2.5-3b


### 🛠️ Solución rápida: Iniciar Foundry Local desde Notebook (Opcional)

Si el diagnóstico anterior muestra que el servicio no está en funcionamiento, puedes intentar iniciarlo desde aquí:

**Nota:** Esto funciona mejor en Windows. En otras plataformas, utiliza comandos en la terminal.


### ⚠️ Solución de problemas de errores de conexión

Si ves `APIConnectionError`, es posible que el servicio Foundry Local no esté funcionando o que los modelos no estén cargados. Prueba estos pasos:

**1. Verificar el estado del servicio:**
```bash
# In a terminal (not in notebook):
foundry service status
```

**2. Iniciar el servicio (si no está funcionando):**
```bash
foundry service start
```

**3. Cargar los modelos necesarios:**
```bash
# Load the models needed for comparison
foundry model run phi-4-mini
foundry model run qwen2.5-7b

# Or use alternative models:
foundry model run phi-3.5-mini
foundry model run qwen2.5-3b
```

**4. Verificar que los modelos estén disponibles:**
```bash
foundry model ls
```

**Problemas comunes:**
- ❌ Servicio no funcionando → Ejecuta `foundry service start`
- ❌ Modelos no cargados → Ejecuta `foundry model run <model-name>`
- ❌ Conflictos de puertos → Verifica si otro servicio está usando el puerto
- ❌ Bloqueo por firewall → Asegúrate de que las conexiones locales estén permitidas

**Solución rápida:** Ejecuta la celda de diagnóstico a continuación antes de la verificación previa.


In [36]:
preflight = {}
retries = 2  # Number of retry attempts

for a in (SLM, LLM):
    mgr, c, mid, info = setup(a, endpoint=ENDPOINT, retries=retries)
    # Keep the original status from info (either 'success' or 'failed')
    preflight[a] = info

print('\n[Pre-flight Check]')
for alias, details in preflight.items():
    status_icon = '✅' if details['status'] == 'success' else '❌'
    print(f"  {status_icon} {alias}: {details['status']} - {details.get('resolved', details.get('error', 'unknown'))}")

preflight

[Init] Connecting to 'phi-4-mini' (attempt 1/2)...
[OK] Connected to 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1

[Pre-flight Check]
  ✅ phi-4-mini: success - Phi-4-mini-instruct-cuda-gpu:4
  ✅ qwen2.5-7b: success - qwen2.5-7b-instruct-cuda-gpu:3


{'phi-4-mini': {'endpoint': 'http://127.0.0.1:59959/v1',
  'resolved': 'Phi-4-mini-instruct-cuda-gpu:4',
  'attempts': 1,
  'status': 'success'},
 'qwen2.5-7b': {'endpoint': 'http://127.0.0.1:59959/v1',
  'resolved': 'qwen2.5-7b-instruct-cuda-gpu:3',
  'attempts': 1,
  'status': 'success'}}

### ✅ Verificación previa: Disponibilidad del modelo

Esta celda verifica que ambos modelos se puedan alcanzar en el punto de conexión configurado antes de ejecutar la comparación.


### Explicación: Comparación de ejecución y recopilación de resultados
Itera sobre ambos alias utilizando el patrón oficial del SDK de Foundry:
1. Inicializa cada modelo con setup() (usa FoundryLocalManager)
2. Ejecuta la inferencia con la API compatible con OpenAI
3. Captura la latencia, los tokens y una muestra de salida
4. Genera un resumen en JSON con un análisis comparativo

Esto sigue el mismo patrón que los ejemplos del taller en session04/model_compare.py.


In [40]:
results = []
retries = 2  # Number of retry attempts

for alias in (SLM, LLM):
    mgr, client, mid, info = setup(alias, endpoint=ENDPOINT, retries=retries)
    if client:
        r = run(client, mid, PROMPT)
        results.append({'alias': alias, **r})
    else:
        # If setup failed, record error
        results.append({
            'alias': alias,
            'status': 'error',
            'error': info.get('error', 'Setup failed'),
            'elapsed_sec': 0,
            'tokens': None,
            'model': alias
        })

# Display results
print(json.dumps(results, indent=2))

# Quick comparative view
print('\n' + '='*80)
print('COMPARISON SUMMARY')
print('='*80)
print(f"{'Alias':<20} {'Status':<15} {'Latency(s)':<15} {'Tokens':<15}")
print('-'*80)

for row in results:
    status = row.get('status', 'unknown')
    status_icon = '✅' if status == 'success' else '❌'
    latency_str = f"{row.get('elapsed_sec', 0):.3f}" if row.get('elapsed_sec') else 'N/A'
    
    # Handle token display - show if available or indicate estimated
    tokens = row.get('tokens')
    usage = row.get('usage', {})
    if tokens:
        if 'estimated_tokens' in usage:
            tokens_str = f"~{tokens} (est.)"
        else:
            tokens_str = str(tokens)
    else:
        tokens_str = 'N/A'
    
    print(f"{status_icon} {row['alias']:<18} {status:<15} {latency_str:<15} {tokens_str:<15}")

print('-'*80)

# Show detailed token breakdown if available
print("\nDetailed Token Usage:")
for row in results:
    if row.get('status') == 'success' and row.get('usage'):
        usage = row['usage']
        print(f"\n  {row['alias']}:")
        if 'prompt_tokens' in usage and usage['prompt_tokens']:
            print(f"    Prompt tokens:     {usage['prompt_tokens']}")
            print(f"    Completion tokens: {usage['completion_tokens']}")
            print(f"    Total tokens:      {usage['total_tokens']}")
        elif 'estimated_tokens' in usage:
            print(f"    Estimated prompt:     {usage['estimated_prompt_tokens']}")
            print(f"    Estimated completion: {usage['estimated_completion_tokens']}")
            print(f"    Estimated total:      {usage['estimated_tokens']}")
            print(f"    (API did not provide token counts - using ~4 chars/token estimate)")

print('\n' + '='*80)

# Calculate speedup if both succeeded
if len(results) == 2 and all(r.get('status') == 'success' and r.get('elapsed_sec') for r in results):
    speedup = results[1]['elapsed_sec'] / results[0]['elapsed_sec']
    print(f"\n💡 SLM is {speedup:.2f}x faster than LLM for this prompt")
    
    # Compare token throughput if available
    slm_tokens = results[0].get('tokens', 0)
    llm_tokens = results[1].get('tokens', 0)
    if slm_tokens and llm_tokens:
        slm_tps = slm_tokens / results[0]['elapsed_sec']
        llm_tps = llm_tokens / results[1]['elapsed_sec']
        print(f"   SLM throughput: {slm_tps:.1f} tokens/sec")
        print(f"   LLM throughput: {llm_tps:.1f} tokens/sec")
        
elif any(r.get('status') == 'error' for r in results):
    print(f"\n⚠️  Some models failed - check errors above")
    print("   Ensure Foundry Local is running: foundry service start")
    print("   Ensure models are loaded: foundry model run <model-name>")

results

[Init] Connecting to 'phi-4-mini' (attempt 1/2)...
[OK] Connected to 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1
[
  {
    "alias": "phi-4-mini",
    "status": "success",
    "content": "1. Reduced Latency: Local AI inference can significantly reduce latency by processing data closer to the source, which is particularly beneficial for real-time applications such as autonomous vehicles or augmented reality.\n\n2. Enhanced Privacy: By keeping data processing local, sensitive information is less likely to be exposed to external networks, thereby enhancing privacy and security.\n\n3. Lower Bandwidth Usage: Local AI inference reduces the n

[{'alias': 'phi-4-mini',
  'status': 'success',
  'content': '1. Reduced Latency: Local AI inference can significantly reduce latency by processing data closer to the source, which is particularly beneficial for real-time applications such as autonomous vehicles or augmented reality.\n\n2. Enhanced Privacy: By keeping data processing local, sensitive information is less likely to be exposed to external networks, thereby enhancing privacy and security.\n\n3. Lower Bandwidth Usage: Local AI inference reduces the need for data transmission over the network, which can save bandwidth and reduce the risk of network congestion.\n\n4. Improved Reliability: Local processing can be more reliable, as it is less dependent on network connectivity. This is particularly important in scenarios where network connectivity is unreliable or intermittent.\n\n5. Scalability: Local AI inference can be easily scaled by adding more local processing units, making it easier to handle increasing data volumes or m

### Interpretación de Resultados

**Métricas Clave:**
- **Latencia**: Menor es mejor - indica un tiempo de respuesta más rápido
- **Tokens**: Mayor rendimiento = más tokens procesados
- **Ruta**: Confirma qué punto de acceso de la API se utilizó

**Cuándo usar SLM vs LLM:**
- **SLM (Modelo de Lenguaje Pequeño)**: Respuestas rápidas, menor uso de recursos, ideal para tareas simples
- **LLM (Modelo de Lenguaje Grande)**: Mayor calidad, mejor razonamiento, úsalo cuando la calidad sea lo más importante

**Próximos Pasos:**
1. Prueba diferentes indicaciones para observar cómo la complejidad afecta la comparación
2. Experimenta con otros pares de modelos
3. Utiliza las muestras del enrutador del taller (Sesión 06) para enrutar inteligentemente según la complejidad de la tarea


In [38]:
# Final Validation Check
print("="*70)
print("VALIDATION SUMMARY")
print("="*70)
print(f"✅ SLM Model: {SLM}")
print(f"✅ LLM Model: {LLM}")
print(f"✅ Using Foundry SDK Pattern: workshop_utils with FoundryLocalManager")
print(f"✅ Pre-flight passed: {all(v['status'] == 'success' for v in preflight.values()) if 'preflight' in dir() else 'Not run yet'}")
print(f"✅ Comparison completed: {len(results) == 2 if 'results' in dir() else 'Not run yet'}")
print(f"✅ Both models responded: {all(r.get('status') == 'success' for r in results) if 'results' in dir() and results else 'Not run yet'}")
print("="*70)

# Check for common configuration issues
issues = []
if 'LLM' in dir() and LLM not in ['qwen2.5-3b', 'qwen2.5-0.5b', 'qwen2.5-1.5b', 'qwen2.5-7b', 'phi-3.5-mini']:
    issues.append(f"⚠️  LLM is '{LLM}' - expected qwen2.5-3b for memory efficiency")
if 'preflight' in dir() and not all(v['status'] == 'success' for v in preflight.values()):
    issues.append("⚠️  Pre-flight check failed - models not accessible")
if 'results' in dir() and results and not all(r.get('status') == 'success' for r in results):
    issues.append("⚠️  Comparison incomplete - check for errors above")

if not issues and 'results' in dir() and results and all(r.get('status') == 'success' for r in results):
    print("🎉 ALL CHECKS PASSED! Notebook completed successfully.")
    print(f"   SLM ({SLM}) vs LLM ({LLM}) comparison completed.")
    if len(results) == 2:
        speedup = results[1]['elapsed_sec'] / results[0]['elapsed_sec'] if results[0]['elapsed_sec'] > 0 else 0
        print(f"   Performance: SLM is {speedup:.2f}x faster")
elif issues:
    print("\n⚠️  Issues detected:")
    for issue in issues:
        print(f"   {issue}")
    print("\n💡 Troubleshooting:")
    print("   1. Ensure service is running: foundry service start")
    print("   2. Load models: foundry model run phi-4-mini && foundry model run qwen2.5-7b")
    print("   3. Check model list: foundry model ls")
else:
    print("\n💡 Run all cells above first, then re-run this validation.")
print("="*70)

VALIDATION SUMMARY
✅ SLM Model: phi-4-mini
✅ LLM Model: qwen2.5-7b
✅ Using Foundry SDK Pattern: workshop_utils with FoundryLocalManager
✅ Pre-flight passed: True
✅ Comparison completed: True
✅ Both models responded: True
🎉 ALL CHECKS PASSED! Notebook completed successfully.
   SLM (phi-4-mini) vs LLM (qwen2.5-7b) comparison completed.
   Performance: SLM is 5.14x faster



---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Aunque nos esforzamos por garantizar la precisión, tenga en cuenta que las traducciones automatizadas pueden contener errores o imprecisiones. El documento original en su idioma nativo debe considerarse como la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que puedan surgir del uso de esta traducción.
